In [1]:
%matplotlib inline
import h5py
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm

# nice plot styling
# plt.style.use('ggplot')
matplotlib.rcParams['font.family'] = 'monospace'
matplotlib.rcParams['font.size'] = 20.0
matplotlib.rcParams['text.usetex'] = False#True
matplotlib.rcParams['figure.figsize'] = (15,10)
matplotlib.rcParams['lines.linewidth'] = 1.0
matplotlib.rcParams['grid.linewidth'] = 1.5

In [5]:
fig = plt.figure()
fig.get_axes
plt.show()

In [2]:
import hydroPlayground as hp
reload(hp);
   
import params
reload(params)

sod_test = hp.HydroProblem(verbose=True)

ngrid = sod_test.nx[sod_test.dim-1]

sod_test.pygrid['rho'][:ngrid/2] = 1.0
sod_test.pygrid['rho'][ngrid/2:] = 0.125
sod_test.pygrid['mom'] = 0.0
sod_test.pygrid['etot'][:ngrid/2] = 1.0
sod_test.pygrid['etot'][ngrid/2:] = 1.0 #25


print sod_test.pygrid['rho']
print sod_test.strides[0]

# sod_test.pygrid[sod]

print sod_test.run()
print sod_test.pygrid['rho']

 eos_gamma = 1.4
 dim = 1
 nx = (256,)
 name = default_hp_problem
 dx = 0.00390625
 cfl_fac = 0.4
 bctype = wall
 init_grid = <function _default_init at 0x7ff890d88e60>
[ 1.     1.     1.     1.     1.     1.     1.     1.     1.     1.     1.
  1.     1.     1.     1.     1.     1.     1.     1.     1.     1.     1.
  1.     1.     1.     1.     1.     1.     1.     1.     1.     1.     1.
  1.     1.     1.     1.     1.     1.     1.     1.     1.     1.     1.
  1.     1.     1.     1.     1.     1.     1.     1.     1.     1.     1.
  1.     1.     1.     1.     1.     1.     1.     1.     1.     1.     1.
  1.     1.     1.     1.     1.     1.     1.     1.     1.     1.     1.
  1.     1.     1.     1.     1.     1.     1.     1.     1.     1.     1.
  1.     1.     1.     1.     1.     1.     1.     1.     1.     1.     1.
  1.     1.     1.     1.     1.     1.     1.     1.     1.     1.     1.
  1.     1.     1.     1.     1.     1.     1.     1.     1.     1.     1.
  1.  

In [132]:
rho_post, rho_mid, p_post, v_post, v_shock = sod_exact(1.0, 1.0, 0.125, 0.1, 1.4)
print rho_post, rho_mid, p_post, v_post, v_shock

0.273939341502 0.442145586428 0.31900053531 0.890952333496 1.63869997736


In [133]:
# iteratively solve the shock jump conditions
def sod_exact(rho_l, p_l, rho_r, p_r, gamma):
    
    Gamma = (gamma - 1.0)/(gamma + 1.0)
    Beta = (gamma - 1.0)/(2.0*gamma)
    
    p_post = p_r # initial guess
    tol = 1.0e-16 # machine precision
    p_post_old = 1.0e99
    
    i = 0
    while abs(1.0 - p_post_old/p_post) > tol and i < 500:
        
        p_post_old = p_post
        p_post = p_r + 2.0*gamma**0.5/(gamma - 1.0)*(1.0 - p_post**Beta) \
            /((1-Gamma)**2/(rho_r*(p_post + Gamma*p_r)))**0.5
        ++i
    v_post = 2.0*gamma**0.5/(gamma - 1.0)*(1.0 - p_post**((gamma - 1)/(2.0*gamma)))
    rho_post = rho_r*((p_post/p_r) + Gamma)/(1 + Gamma*(p_post/p_r))
    v_shock = v_post*(rho_post/rho_r)/((rho_post/rho_r) - 1)
    rho_mid = rho_l*(p_post/p_l)**(1.0/gamma)
    return rho_post, rho_mid, p_post, v_post, v_shock

def plot_sod(ax_rho, ax_p, ax_u, ax_v, rho_l, p_l, rho_r, p_r, gamma, x0, t):
    rho_post, rho_mid, p_post, v_post, v_shock = sod_exact(rho_l, p_l, rho_r, p_r, gamma)
    
    u_post = p_post/((gamma - 1)*rho_post)
    u_mid = p_post/((gamma - 1)*rho_mid)
    u_l = p_l/((gamma - 1)*rho_l)
    u_r = p_r/((gamma - 1)*rho_r)
    
    m2 = (gamma - 1.0)/(gamma + 1.0)
    c_left = (gamma*p_l/rho_l)**0.5
    
    x_shock = x0 + v_shock*t
    x_contact = x0 + v_post*t
    
    x_rare_l = x0 - c_left*t
    x_rare_r = x0 - t*(c_left - v_post/(1.0 - m2))
    x_rare_sp = np.linspace(x_rare_l, x_rare_r, 100)
    
    c_rare = m2*(x0 - x_rare_sp)/t + (1.0 - m2)*c_left
    rho_rare = rho_l*(c_rare/c_left)**(2.0/(gamma - 1))
    p_rare = p_l*(rho_rare/rho_l)**gamma
    v_rare = (1.0 - m2)*((x_rare_sp - x0)/t + c_left)
    u_rare = p_rare/((gamma - 1)*rho_rare)
    
    marker = 'r'
    key, = ax_rho.plot([0, x_rare_l], [rho_l, rho_l], marker)
    ax_rho.plot([x_shock, 1.0], [rho_r, rho_r], marker)
    ax_rho.plot([x_contact, x_shock], [rho_post, rho_post], marker)
    ax_rho.plot([x_shock, x_shock], [rho_r, rho_post], marker)
    ax_rho.plot([x_contact, x_contact], [rho_post, rho_mid], marker)
    ax_rho.plot([x_rare_r, x_contact], [rho_mid, rho_mid], marker)
    ax_rho.plot(x_rare_sp, rho_rare, marker)
    
    ax_p.plot([0, x_rare_l], [p_l, p_l], marker)
    ax_p.plot([x_shock, 1.0], [p_r, p_r], marker)
    ax_p.plot([x_shock, x_shock], [p_r, p_post], marker)
    ax_p.plot([x_rare_r, x_shock], [p_post, p_post], marker)
    ax_p.plot(x_rare_sp, p_rare, marker)
    
#     ax_u.plot([0, x_rare_l], [u_l, u_l], marker)
#     ax_u.plot([x_shock, 1.0], [u_r, u_r], marker)
#     ax_u.plot([x_contact, x_shock], [u_post, u_post], marker)
#     ax_u.plot([x_shock, x_shock], [u_r, u_post], marker)
#     ax_u.plot([x_contact, x_contact], [u_post, u_mid], marker)
#     ax_u.plot([x_rare_r, x_contact], [u_mid, u_mid], marker)
#     ax_u.plot(x_rare_sp, u_rare, marker)
    
    ax_v.plot([0, x_rare_l], [0, 0], marker)
    ax_v.plot([x_shock, 1.0], [0, 0], marker)
    ax_v.plot([x_shock, x_shock], [0, v_post], marker)
    ax_v.plot([x_rare_r, x_shock], [v_post, v_post], marker)
    ax_v.plot(x_rare_sp, v_rare, marker)
    
    return key